In [1]:
!pip3 install pyspark

In [2]:
from pyspark import SparkContext

In [49]:
sc.stop()

In [50]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Spark dataframe") \
    .getOrCreate()
sc = spark.sparkContext


In [51]:
spark

### DataFrames

DataFrames gives a schema view of data basically, it is an abstraction. In dataframes, view of data is organized as columns with column name and types info. In addition, we can say data in dataframe is as same as the table in relational database.

### DataSets

In Spark, datasets are an extension of dataframes. Basically, it earns two different APIs characteristics, such as strongly typed and untyped. Datasets are by default a collection of strongly typed JVM objects

**data frames** allow to deal with data in structured way. We will see how the data frame abstraction, very popular in other data analytics ecosystems (e.g. R and Python/Pandas), it is very powerful when performing exploratory data analysis. In fact, it is very easy to express data queries when used together with the SQL language

#### Creating a RDD from a file
The most common way of creating an RDD is to load it from a file. Notice that Spark's textFile can handle compressed files directly.

In [52]:
!wget -c https://s3.eu-central-1.amazonaws.com/jkan.pl/data-science/uek/otomoto.csv

--2020-12-12 01:03:30--  https://s3.eu-central-1.amazonaws.com/jkan.pl/data-science/uek/otomoto.csv
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.72.175
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.72.175|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [53]:
raw_data = sc.textFile('otomoto.csv')
csv_data = raw_data.map(lambda l: l.split(";"))
header = csv_data.first() #extract header
data = csv_data.filter(lambda r: r != header)
data.cache()

PythonRDD[3] at RDD at PythonRDD.scala:53

In [54]:
data.take(2)

[['"6039269187"',
  '"https://www.otomoto.pl/oferta/renault-fluence-bezwypadkowy-instalacja-gazowa-ID6AI9Nx.html"',
  '""',
  '"Renault"',
  '"Fluence"',
  '"Osobowe"',
  '"1 600 cm3"',
  '"5"',
  '""',
  '""',
  '"Sedan"',
  '"2013"',
  '""',
  '""',
  '""',
  '""',
  '""',
  '"Tak"',
  '""',
  '"Benzyna+LPG"',
  '""',
  '"Czechy"',
  '""',
  '"Tak"',
  '"Używane"',
  '"8 listopad 2013"',
  '"Renault Fluence"',
  '"25900.0"',
  '"Manualna"',
  '""',
  '""',
  '"4"',
  '""',
  '"172 000 km"',
  '"Tak"',
  '""',
  '""',
  '"Osoby prywatnej"',
  '""',
  '"Biały"',
  '"Na przednie koła"',
  '"110 KM"'],
 ['"6034090022"',
  '"https://www.otomoto.pl/oferta/mercedes-benz-klasa-g-350d-pakiet-amg-designo-manufaktur-edition-ID6AmqsC.html"',
  '"Tak"',
  '"Mercedes-Benz"',
  '"Klasa G"',
  '"Osobowe"',
  '"2 987 cm3"',
  '"5"',
  '"Tak"',
  '"W463 (1992-)"',
  '"SUV"',
  '"2016"',
  '"Tak"',
  '""',
  '"WDB4633481X263455"',
  '""',
  '""',
  '"Tak"',
  '"Tak"',
  '"Diesel"',
  '"261 g/km"',
  '"

In [55]:
list(enumerate(header))

[(0, '"id"'),
 (1, '"url"'),
 (2, '"Faktura VAT"'),
 (3, '"Marka pojazdu"'),
 (4, '"Model pojazdu"'),
 (5, '"Kategoria"'),
 (6, '"Pojemność skokowa"'),
 (7, '"Liczba miejsc"'),
 (8, '"Zarejestrowany w Polsce"'),
 (9, '"Wersja"'),
 (10, '"Typ"'),
 (11, '"Rok produkcji"'),
 (12, '"Serwisowany w ASO"'),
 (13, '"Leasing"'),
 (14, '"VIN"'),
 (15, '"Homologacja ciężarowa"'),
 (16, '"VAT marża"'),
 (17, '"Pierwszy właściciel"'),
 (18, '"Filtr cząstek stałych"'),
 (19, '"Rodzaj paliwa"'),
 (20, '"Emisja CO2"'),
 (21, '"Kraj pochodzenia"'),
 (22, '"Numer rejestracyjny pojazdu"'),
 (23, '"Bezwypadkowy"'),
 (24, '"Stan"'),
 (25, '"Pierwsza rejestracja"'),
 (26, '"title"'),
 (27, '"price"'),
 (28, '"Skrzynia biegów"'),
 (29, '"Perłowy"'),
 (30, '"Kierownica po prawej (Anglik)"'),
 (31, '"Liczba drzwi"'),
 (32, '"Metalik"'),
 (33, '"Przebieg"'),
 (34, '"Akryl (niemetalizowany)"'),
 (35, '"Możliwość finansowania"'),
 (36, '"Uszkodzony"'),
 (37, '"Oferta od"'),
 (38, '"Kod Silnika"'),
 (39, '"Kolor"'

In [73]:
from pyspark.sql import Row
row_data = data\
    .map(lambda p: Row(
        _id=(p[0]),
        marka=(p[3]),
        model=(p[4]),
        price=(p[27]),
        przebieg=(p[33])
    )
)

Once we have our RDD of Row we can infer and register the schema.

In [74]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [75]:
DF = sqlContext.createDataFrame(row_data)

In [76]:
DF.take(1)

[Row(_id='"6039269187"', marka='"Renault"', model='"Fluence"', price='"25900.0"', przebieg='"172 000 km"')]

In [81]:
DF.select("marka", "model", "przebieg", "price")\
    .write\
    .format('csv') \
    .mode("overwrite") \
    .options(delimiter=";")\
    .save("adv_model_przebieg")

In [82]:
import pandas as pd

In [86]:
import glob
all_files = glob.glob('./adv_model_przebieg/*.csv')
df_from_each_file = (pd.read_csv(f, delimiter=";") for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [87]:
concatenated_df

,"\172 000 km\""""","\25900.0\""""","\33 400 km\""""","\99900.0\""""","\Fluence\""""","\Mercedes-Benz\""""","\Renault\""""","\Vito\"""""
0,"\52 904 km\""""","\434900.0\""""",NaN,NaN,"\Klasa G\""""",NaN,"\Mercedes-Benz\""""",NaN
1,"\56 000 km\""""","\41900.0\""""",NaN,NaN,"\A5\""""",NaN,"\Audi\""""",NaN
2,"\88 000 km\""""","\14900.0\""""",NaN,NaN,"\Polo\""""",NaN,"\Volkswagen\""""",NaN
3,"\176 000 km\""""","\5900.0\""""",NaN,NaN,"\206 CC\""""",NaN,"\Peugeot\""""",NaN
4,"\300 000 km\""""","\7900.0\""""",NaN,NaN,"\Bora\""""",NaN,"\Volkswagen\""""",NaN
5,"\301 000 km\""""","\9800.0\""""",NaN,NaN,"\Seria 5\""""",NaN,"\BMW\""""",NaN
6,"\155 400 km\""""","\98900.0\""""",NaN,NaN,"\Q5\""""",NaN,"\Audi\""""",NaN
7,"\124 500 km\""""","\14480.0\""""",NaN,NaN,"\Santa Fe\""""",NaN,"\Hyundai\""""",NaN
8,"\95 100 km\""""","\64900.0\""""",NaN,NaN,"\Golf\""""",NaN,"\Volkswagen\""""",NaN
9,"\268 000 km\""""","\37000.0\""""",NaN,NaN,"\Grand Cherokee\""""",NaN,"\Jeep\""""",NaN


https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html

In [88]:
DF = spark.read.load("otomoto.csv", format="csv", sep=";", inferSchema="true", header="true")

In [89]:
DF.take(1)

[Row(id=6039269187, url='https://www.otomoto.pl/oferta/renault-fluence-bezwypadkowy-instalacja-gazowa-ID6AI9Nx.html', Faktura VAT=None, Marka pojazdu='Renault', Model pojazdu='Fluence', Kategoria='Osobowe', Pojemność skokowa='1 600 cm3', Liczba miejsc=5, Zarejestrowany w Polsce=None, Wersja=None, Typ='Sedan', Rok produkcji=2013, Serwisowany w ASO=None, Leasing=None, VIN=None, Homologacja ciężarowa=None, VAT marża=None, Pierwszy właściciel='Tak', Filtr cząstek stałych=None, Rodzaj paliwa='Benzyna+LPG', Emisja CO2=None, Kraj pochodzenia='Czechy', Numer rejestracyjny pojazdu=None, Bezwypadkowy='Tak', Stan='Używane', Pierwsza rejestracja='8 listopad 2013', title='Renault Fluence', price=25900.0, Skrzynia biegów='Manualna', Perłowy=None, Kierownica po prawej (Anglik)=None, Liczba drzwi=4, Metalik=None, Przebieg='172 000 km', Akryl (niemetalizowany)='Tak', Możliwość finansowania=None, Uszkodzony=None, Oferta od='Osoby prywatnej', Kod Silnika=None, Kolor='Biały', Napęd='Na przednie koła', Moc

In [90]:
type(DF)

pyspark.sql.dataframe.DataFrame

In [91]:
DF.registerTempTable("cars")

In [92]:
volvos = sqlContext.sql("""
    Select `Marka pojazdu` as marka, `Model pojazdu` as model, `price`, `przebieg`, `Rok produkcji` as rok  from cars 
    WHERE `Marka pojazdu` = "Volvo"
    AND `Model pojazdu` = "XC 60"
""")

In [93]:
type(volvos)

pyspark.sql.dataframe.DataFrame

The results of SQL queries are RDDs and support all the normal RDD operations.

In [94]:
volvos.take(1)

[Row(marka='Volvo', model='XC 60', price=89900.0, przebieg='132 789 km', rok=2016)]

In [95]:
volvos.show()

+-----+-----+--------+----------+----+
|marka|model|   price|  przebieg| rok|
+-----+-----+--------+----------+----+
|Volvo|XC 60| 89900.0|132 789 km|2016|
|Volvo|XC 60|121951.0| 25 236 km|2016|
|Volvo|XC 60| 54900.0|221 000 km|2012|
|Volvo|XC 60| 79999.0|140 515 km|2012|
|Volvo|XC 60| 89900.0|159 900 km|2012|
|Volvo|XC 60| 98900.0| 22 739 km|2016|
|Volvo|XC 60| 79555.0| 62 321 km|2012|
|Volvo|XC 60| 52900.0| 60 000 km|2014|
|Volvo|XC 60| 66900.0| 15 000 km|2015|
|Volvo|XC 60| 59900.0|183 000 km|2010|
|Volvo|XC 60|112800.0| 32 000 km|2016|
|Volvo|XC 60|123000.0| 21 000 km|2016|
|Volvo|XC 60| 87000.0|144 000 km|2015|
|Volvo|XC 60| 88900.0| 66 200 km|2014|
|Volvo|XC 60|185900.0|  3 000 km|2017|
|Volvo|XC 60| 75900.0|132 000 km|2014|
|Volvo|XC 60| 43500.0|205 500 km|2010|
|Volvo|XC 60| 68000.0|110 094 km|2014|
|Volvo|XC 60|122900.0| 13 000 km|2017|
|Volvo|XC 60| 79900.0|145 000 km|2016|
+-----+-----+--------+----------+----+
only showing top 20 rows



In [96]:
for volvo in volvos.collect():
    print("{} for {} with millage {}".format(volvo.model, volvo.price, volvo.przebieg))

XC 60 for 89900.0 with millage 132 789 km
XC 60 for 121951.0 with millage 25 236 km
XC 60 for 54900.0 with millage 221 000 km
XC 60 for 79999.0 with millage 140 515 km
XC 60 for 89900.0 with millage 159 900 km
XC 60 for 98900.0 with millage 22 739 km
XC 60 for 79555.0 with millage 62 321 km
XC 60 for 52900.0 with millage 60 000 km
XC 60 for 66900.0 with millage 15 000 km
XC 60 for 59900.0 with millage 183 000 km
XC 60 for 112800.0 with millage 32 000 km
XC 60 for 123000.0 with millage 21 000 km
XC 60 for 87000.0 with millage 144 000 km
XC 60 for 88900.0 with millage 66 200 km
XC 60 for 185900.0 with millage 3 000 km
XC 60 for 75900.0 with millage 132 000 km
XC 60 for 43500.0 with millage 205 500 km
XC 60 for 68000.0 with millage 110 094 km
XC 60 for 122900.0 with millage 13 000 km
XC 60 for 79900.0 with millage 145 000 km
XC 60 for 94900.0 with millage 124 999 km
XC 60 for 54000.0 with millage 290 000 km
XC 60 for 71900.0 with millage 97 100 km
XC 60 for 87500.0 with millage 163 000 km

In [97]:
DF.printSchema()

root
 |-- id: long (nullable = true)
 |-- url: string (nullable = true)
 |-- Faktura VAT: string (nullable = true)
 |-- Marka pojazdu: string (nullable = true)
 |-- Model pojazdu: string (nullable = true)
 |-- Kategoria: string (nullable = true)
 |-- Pojemność skokowa: string (nullable = true)
 |-- Liczba miejsc: integer (nullable = true)
 |-- Zarejestrowany w Polsce: string (nullable = true)
 |-- Wersja: string (nullable = true)
 |-- Typ: string (nullable = true)
 |-- Rok produkcji: integer (nullable = true)
 |-- Serwisowany w ASO: string (nullable = true)
 |-- Leasing: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Homologacja ciężarowa: string (nullable = true)
 |-- VAT marża: string (nullable = true)
 |-- Pierwszy właściciel: string (nullable = true)
 |-- Filtr cząstek stałych: string (nullable = true)
 |-- Rodzaj paliwa: string (nullable = true)
 |-- Emisja CO2: string (nullable = true)
 |-- Kraj pochodzenia: string (nullable = true)
 |-- Numer rejestracyjny poja

### Queries as DataFrame operations


In [98]:
DF.select("Marka pojazdu").groupBy("Marka pojazdu").count().show()

+-------------+-----+
|Marka pojazdu|count|
+-------------+-----+
|   Volkswagen| 5598|
|   Oldsmobile|    1|
|     Infiniti|   91|
|      Peugeot| 1906|
|          NSU|    1|
|        Lexus|  252|
|       Jaguar|  215|
|       Saturn|    1|
|          Żuk|    1|
|     Maserati|   43|
|        Rover|   32|
|        Aixam|   37|
|         Tata|    2|
|         Jeep|  426|
|       Lancia|   64|
|   Mitsubishi|  533|
|         Mini|  495|
|          Kia| 1061|
|         null|  470|
|    Chevrolet|  352|
+-------------+-----+
only showing top 20 rows



In [99]:
DF.select(DF["Model pojazdu"]) \
    .filter(DF["Marka pojazdu"]=="Volvo") \
    .groupBy("Model pojazdu") \
    .count().show()

+-------------+-----+
|Model pojazdu|count|
+-------------+-----+
|          S60|  130|
|          V90|   31|
|          850|    4|
|    Seria 400|    1|
|          S80|   59|
|          V60|  112|
|          745|    1|
|          S90|   48|
|          C30|   45|
|          V70|   74|
|        XC 40|   10|
|          V50|  147|
|          S40|   73|
|        XC 70|   37|
|    Seria 700|    1|
|        XC 60|  219|
|         Inny|    1|
|          C70|   10|
|    Seria 200|    1|
|          V40|  151|
+-------------+-----+
only showing top 20 rows

